# [626. Exchange Seats](https://leetcode.com/problems/exchange-seats/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Seat

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| student     | varchar |
+-------------+---------+</pre>
id is the primary key (unique value) column for this table.
Each row of this table indicates the name and the ID of a student.
id is a continuous increment.


Write a solution to swap the seat id of every two consecutive students. If the number of students is odd, the id of the last student is not swapped.

Return the result table ordered by id in ascending order.

The result format is in the following example.



Example 1:

Input:
Seat table:
<pre>+----+---------+
| id | student |
+----+---------+
| 1  | Abbot   |
| 2  | Doris   |
| 3  | Emerson |
| 4  | Green   |
| 5  | Jeames  |
+----+---------+</pre>
Output:
<pre>+----+---------+
| id | student |
+----+---------+
| 1  | Doris   |
| 2  | Abbot   |
| 3  | Green   |
| 4  | Emerson |
| 5  | Jeames  |
+----+---------+</pre>
Explanation:
Note that if the number of students is odd, there is no need to change the last one's seat.

In [3]:
#pandas schema

import pandas as pd

data = [[1, 'Abbot'], [2, 'Doris'], [3, 'Emerson'], [4, 'Green'], [5, 'Jeames']]
seat = pd.DataFrame(data, columns=['id', 'student']).astype({'id': 'Int64', 'student': 'object'})


#to spark dataframe

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

seat_df = spark.createDataFrame(seat)
seat_df.show()

+---+-------+
| id|student|
+---+-------+
|  1|  Abbot|
|  2|  Doris|
|  3|Emerson|
|  4|  Green|
|  5| Jeames|
+---+-------+



In [10]:
l_id = seat_df.agg(F.max('id')).collect()[0][0]

seat_df.withColumn('id',
                   when(col('id')%2==0,expr('id-1'))\
                  .when(col('id')==l_id,col('id'))\
                  .otherwise(expr('id+1')))\
                  .orderBy('id')\
                  .show()

+---+-------+
| id|student|
+---+-------+
|  1|  Doris|
|  2|  Abbot|
|  3|  Green|
|  4|Emerson|
|  5| Jeames|
+---+-------+



In [11]:
# In spark SQL

seat_df.createOrReplaceTempView('seat')

spark.sql('''
WITH last_seat AS (
    SELECT MAX(id) as l_id
    FROM seat
)
select
    case
    when id%2=0 then id-1
    when id = l_id then id
    else id+1
    end as id,
    student
from seat,last_seat
order by id
''').show()

+---+-------+
| id|student|
+---+-------+
|  1|  Doris|
|  2|  Abbot|
|  3|  Green|
|  4|Emerson|
|  5| Jeames|
+---+-------+



In [ ]:
# using subquery in spark sql

spark.sql('''
select
    case
        when id%2=0 then id-1
        when id=(select max(id) from seat) then id
        else id+1 end as id,
    student
from seat
order by id;
''').show()

In [ ]:
spark.stop()